# Betweenness centrality vs threshold and environmental drivers

This notebook computes:
1) Mean Betweenness_Centrality vs threshold (th = 0..34).
2) Betweenness_Centrality response curves vs selected variables


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip -q install rasterio whitebox

In [ ]:
import os
import numpy as np
import pandas as pd
import rasterio
import networkx as nx
import matplotlib.pyplot as plt
import whitebox
from tqdm import tqdm

## Config

In [ ]:
DRIVE_ROOT = '/content/drive/My Drive'
PROJECT_DIR = os.path.join(DRIVE_ROOT, 'Oyster_Flow')
RESULT_DIR = os.path.join(PROJECT_DIR, 'OFG_result')
os.makedirs(RESULT_DIR, exist_ok=True)
# Hydrology rasters
flow_accum = os.path.join(PROJECT_DIR, 'FlowAcc_Flow51.tif')
flow_dir = os.path.join(PROJECT_DIR, 'FlowDir_Fill31.tif')
streams_rast = os.path.join(RESULT_DIR, 'HS_slope_streams.tif')
stream_link_r = os.path.join(RESULT_DIR, 'HS_slope_link.tif')
stream_order_r = os.path.join(RESULT_DIR, 'HS_slope_order.tif')
# Env layers
base_dir = os.path.join(PROJECT_DIR, 'OFG_New')
env_vars = ['w1', 'v1', 'temp1', 'NO32', 'chlorophyll1', 'salt1']
# Thresholds
THRESHOLDS = list(range(35))
wbt = whitebox.WhiteboxTools()
wbt.set_working_dir(RESULT_DIR)


## Helpers

In [ ]:
def load_env_arrays(var_list, base_directory):
    arrays = {}
    for v in var_list:
        path = os.path.join(base_directory, f'{v}.tif')
        with rasterio.open(path) as src:
            arr = src.read(1).astype(np.float32)
            if src.nodata is not None:
                arr = np.where(arr == src.nodata, np.nan, arr)
        arrays[v] = arr
    return arrays

def extract_network(threshold):
    wbt.extract_streams(flow_accum, streams_rast, threshold)
    wbt.stream_link_class(d8_pntr=flow_dir, streams=streams_rast, output=stream_link_r)
    wbt.strahler_stream_order(d8_pntr=flow_dir, streams=streams_rast, output=stream_order_r)

    with rasterio.open(streams_rast) as src:
        streams_data = src.read(1).astype(np.float32)
    with rasterio.open(flow_accum) as src:
        flow_acc_data = src.read(1).astype(np.float32)
    with rasterio.open(stream_order_r) as src:
        stream_order_data = src.read(1).astype(np.float32)

    flow_acc_data[flow_acc_data < 0] = 0

    graph = nx.Graph()
    rows, cols = np.where(streams_data > 0)
    for r, c in zip(rows, cols):
        graph.add_node((r, c), flow_acc=flow_acc_data[r, c], stream_order=stream_order_data[r, c])
    for r, c in zip(rows, cols):
        for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nr, nc = r + dr, c + dc
            if 0 <= nr < streams_data.shape[0] and 0 <= nc < streams_data.shape[1]:
                if streams_data[nr, nc] > 0:
                    graph.add_edge((r, c), (nr, nc))
    return graph

def betweenness_mean(graph):
    if graph.number_of_nodes() == 0:
        return 0.0
    bc = nx.betweenness_centrality(graph)
    return float(np.mean(list(bc.values())))

def node_dataframe(graph, env_arrays):
    bc = nx.betweenness_centrality(graph)
    rows = []
    for (r, c), b in bc.items():
        row = {'Betweenness_Centrality': b}
        for v, arr in env_arrays.items():
            try:
                row[v] = float(arr[r, c])
            except Exception:
                row[v] = np.nan
        rows.append(row)
    df = pd.DataFrame(rows)
    return df.dropna()

def binned_curve(df, x_col, y_col, n_bins=20):
    x = df[x_col].values
    y = df[y_col].values
    mask = np.isfinite(x) & np.isfinite(y)
    x = x[mask]
    y = y[mask]
    if x.size == 0:
        return np.array([]), np.array([])
    bins = np.linspace(np.min(x), np.max(x), n_bins + 1)
    inds = np.digitize(x, bins)
    centers = []
    means = []
    for i in range(1, len(bins)):
        sel = y[inds == i]
        if sel.size > 0:
            centers.append((bins[i-1] + bins[i]) / 2)
            means.append(np.mean(sel))
    return np.array(centers), np.array(means)


## Betweenness_Centrality vs env variables at th=0 and th=15

In [ ]:
env_arrays = load_env_arrays(env_vars, base_dir)
display_names = {'NO32': 'NO$_3$2'}
for th in [0, 15]:
    g = extract_network(th)
    df_nodes = node_dataframe(g, env_arrays)
    fig, axes = plt.subplots(2, 3, figsize=(12, 7), sharey=True)
    axes = axes.flatten()
    for i, var in enumerate(env_vars):
        ax = axes[i]
        x, y = binned_curve(df_nodes, var, 'Betweenness_Centrality', n_bins=20)
        ax.plot(x, y, marker='o', markersize=3)
        ax.set_title(display_names.get(var, var))
        ax.set_xlabel(display_names.get(var, var))
        if i % 3 == 0:
            ax.set_ylabel('Betweenness_Centrality')
        ax.grid(True)
    plt.suptitle(f'Betweenness_Centrality vs env vars (th={th})')
    plt.tight_layout()
    out_pdf = os.path.join(RESULT_DIR, f'betweenness_vs_env_th_{th}.pdf')
    plt.savefig(out_pdf, bbox_inches='tight')
    plt.show()
